In [1]:
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image


C:\Users\Moadh\anaconda3\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 'Could not find module 'C:\Users\Moadh\anaconda3\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
# Paramètres du programme
log_interval = 100
no_cuda = False
no_mps = False

# Paramètres du modèle et de l'algorithme
batch_size = 128
Nepochs = 10
int_dim = 400 
latent_dim = 20
p = 28
img_dim = p**2

cuda = not no_cuda and torch.cuda.is_available()
use_mps = not no_mps and torch.backends.mps.is_available()

torch.manual_seed(200)

if cuda:
    device = torch.device("cuda")
elif use_mps:
    device = torch.device("mps")
else:
    device = torch.device("cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if not cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=False, **kwargs)

In [3]:
# Implémentation du VAE

class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        # Encodeur
        self.fc1 = nn.Linear(img_dim, int_dim)
        self.do1 = nn.Dropout(0.1)
        self.fc2 = nn.Linear(int_dim, int_dim)
        self.do2 = nn.Dropout(0.1)
        self.fc31 = nn.Linear(int_dim, latent_dim)
        self.fc32 = nn.Linear(int_dim, latent_dim)
        self.sp = nn.Softplus()
        # Décodeur
        self.fc4 = nn.Linear(latent_dim, int_dim)
        self.do3 = nn.Dropout(0.1)
        self.fc5 = nn.Linear(int_dim, int_dim)
        self.do4 = nn.Dropout(0.1)        
        self.fc6 = nn.Linear(int_dim, img_dim)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        h1 = self.do1(h1)
        h1 = F.relu(self.fc2(h1))
        h1 = self.do2(h1)
        return self.fc31(h1), self.fc32(h1)

    def reparameterize(self, mu, logvar):
        std = 1e-6 + self.sp(logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc4(z))
        h3 = self.do3(h3)
        h3 = F.relu(self.fc5(h3))
        h3 = self.do4(h3)
        return torch.sigmoid(self.fc6(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        xhat = self.decode(z)
        return mu, logvar, xhat


model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [4]:
# Fonction de perte
def loss_function(x, xhat, mu, logvar):
        
    # log p(x|z) 
    BCE = F.binary_cross_entropy(xhat, x.view(-1, 784), reduction='sum')

    # Divergence KL
    KLD = 0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE - KLD


def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        mu, logvar, xhat = model(data)
        loss = loss_function(data, xhat, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            mu, logvar, xhat = model(data)
            test_loss += loss_function(data, xhat, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      xhat.view(batch_size, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [5]:

for epoch in range(1, Nepochs + 1):
    train(epoch)
    test(epoch)
    with torch.no_grad():
        sample = torch.randn(64, latent_dim).to(device)
        xhat = model.decode(sample)
        save_image(xhat.cpu().view(64, 1, p, p),
                    'results/sample_' + str(epoch) + '.png')


Train Epoch: 1 [0/60000 (0%)]	Loss: 544.292114
Train Epoch: 1 [12800/60000 (21%)]	Loss: 191.585876
Train Epoch: 1 [25600/60000 (43%)]	Loss: 171.737198
Train Epoch: 1 [38400/60000 (64%)]	Loss: 152.920044
Train Epoch: 1 [51200/60000 (85%)]	Loss: 138.237778
====> Epoch: 1 Average loss: 174.4666
====> Test set loss: 131.1991
Train Epoch: 2 [0/60000 (0%)]	Loss: 135.883713
Train Epoch: 2 [12800/60000 (21%)]	Loss: 126.653244
Train Epoch: 2 [25600/60000 (43%)]	Loss: 127.084381
Train Epoch: 2 [38400/60000 (64%)]	Loss: 120.384583
Train Epoch: 2 [51200/60000 (85%)]	Loss: 118.324219
====> Epoch: 2 Average loss: 125.0473
====> Test set loss: 110.9808
Train Epoch: 3 [0/60000 (0%)]	Loss: 120.965546
Train Epoch: 3 [12800/60000 (21%)]	Loss: 112.059715
Train Epoch: 3 [25600/60000 (43%)]	Loss: 114.972801
Train Epoch: 3 [38400/60000 (64%)]	Loss: 112.432327
Train Epoch: 3 [51200/60000 (85%)]	Loss: 110.167137
====> Epoch: 3 Average loss: 113.8438
====> Test set loss: 103.9826
Train Epoch: 4 [0/60000 (0%)]	L